In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
import gc
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
import joblib
tqdm.pandas()
pd.set_option("display.max_columns", None)

train_df = pd.read_parquet("task3_recsys/train_df_after_transformer.parquet")
test_df = pd.read_parquet("task3_recsys/test_df_after_transformer.parquet")
numeric_df = pd.read_parquet("task3_recsys/alfabattle_df4transformer_feb7_v2.parquet")
numeric_df = numeric_df.rename(columns={"last_mcc_category": "next_mcc"})

Сделаем признаки для бустинга, помимо эмбеддингов трансформеров.

In [2]:
numeric_df["top5_most_common_mcc"] = numeric_df.mcc_cat_list.progress_apply(lambda x: Counter(x).most_common(5))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 963811/963811 [00:16<00:00, 57709.27it/s]


In [3]:
numeric_df["days_b_list_std"] = numeric_df.days_b_list.map(np.std)
numeric_df["days_b_list_mean"] = numeric_df.days_b_list.map(np.mean)
numeric_df["days_b_list_min"] = numeric_df.days_b_list.map(min)
numeric_df["days_b_list_max"] = numeric_df.days_b_list.map(max)
numeric_df["total_trx"] = numeric_df.mcc_cat_list.map(len)
numeric_df = numeric_df.drop(columns=["days_b_list"])

In [4]:
for i in tqdm(range(1, 6)):
    i_real = i - 1
    numeric_df[f"mcc_top{i}_class"] = numeric_df.top5_most_common_mcc.apply(lambda x: x[i_real][0] if len(x) > i_real else -1)
    numeric_df[f"mcc_top{i}_counts"] = numeric_df.top5_most_common_mcc.apply(lambda x: x[i_real][1] if len(x) > i_real else -1)

    numeric_df[f"amnt_temp_{i}"] = numeric_df.apply(lambda row: row.amnt_list[(row.mcc_cat_list == row[f"mcc_top{i}_class"])] if row[f"mcc_top{i}_class"] else [], axis=1)

    numeric_df[f"mcc_top{i}_amount_mean"] = numeric_df.apply(lambda row: row[f"amnt_temp_{i}"].mean() if len(row[f"amnt_temp_{i}"]) else -1, axis=1)
    numeric_df[f"mcc_top{i}_amount_std"] = numeric_df.apply(lambda row: row[f"amnt_temp_{i}"].std() if len(row[f"amnt_temp_{i}"]) else -1, axis=1)
    numeric_df[f"mcc_top{i}_amount_max"] = numeric_df.apply(lambda row: row[f"amnt_temp_{i}"].max() if len(row[f"amnt_temp_{i}"]) else -1, axis=1)
    numeric_df[f"mcc_top{i}_amount_min"] = numeric_df.apply(lambda row: row[f"amnt_temp_{i}"].min() if len(row[f"amnt_temp_{i}"]) else -1, axis=1)

numeric_df = numeric_df.drop(columns=[f"amnt_temp_{i}" for i in range(1, 6)])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:51<00:00, 34.28s/it]


In [5]:
numeric_df["last_mcc"] = numeric_df.mcc_cat_list.apply(lambda x: x[-1])
numeric_df["amnt_list_temp"] = numeric_df.amnt_list.apply(lambda x: x[np.nonzero(x)])

numeric_df["amnt_list_std"] = numeric_df.amnt_list_temp.apply(lambda x: np.std(x) if len(x) else -1)
numeric_df["amnt_list_mean"] = numeric_df.amnt_list_temp.apply(lambda x: np.mean(x) if len(x) else -1)
numeric_df["amnt_list_min"] = numeric_df.amnt_list_temp.apply(lambda x: np.min(x) if len(x) else -1)
numeric_df["amnt_list_max"] = numeric_df.amnt_list_temp.apply(lambda x: np.max(x) if len(x) else -1)

In [6]:
numeric_df = numeric_df.drop(columns=["amnt_list_temp", "mcc_cat_list", "amnt_list", "top5_most_common_mcc"])

К сожалению, explode (для применения векторизованных операций Pandas) сделать не получится из-за ограничений в ОЗУ, поэтому применяем вложенные циклы :):

In [7]:
topk_cols = ["city_list", "op_kind_list", "card_type_list", "op_type_list", "payment_system_list", "income_list"]

gc.collect()

for col in tqdm(topk_cols):
    numeric_df[f"top5_{col[:-5]}"] = numeric_df[col].apply(lambda x: Counter(x).most_common(5))

for i in tqdm(range(1, 6)):
    i_real = i - 1
    for col_k in topk_cols:
        col_k_s = col_k[:-5]
        numeric_df[f"{col_k_s}_top{i}_class"] = numeric_df[f"top5_{col_k_s}"].apply(lambda x: x[i_real][0] if len(x) > i_real else -1)
        numeric_df[f"{col_k_s}_top{i}_counts"] = numeric_df[f"top5_{col_k_s}"].apply(lambda x: x[i_real][1] if len(x) > i_real else -1)
        if i == 5:
            numeric_df[f"last_{col_k_s}"] = numeric_df[col_k].apply(lambda x: x[-1])

numeric_df = numeric_df.drop(columns=topk_cols + [i for i in numeric_df.columns if i.startswith("top5")])

 80%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                        | 4/5 [00:43<00:09,  9.97s/it]/tmp/ipykernel_60334/2695031921.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  numeric_df[f"{col_k_s}_top{i}_counts"] = numeric_df[f"top5_{col_k_s}"].apply(lambda x: x[i_real][1] if len(x) > i_real else -1)
/tmp/ipykernel_60334/2695031921.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  

In [8]:
numeric_df

,app_id,next_mcc,last_card_type,last_week,ecomm_1,ecomm_2,optype_gr_1,optype_gr_2,optype_gr_3,days_b_list_std,days_b_list_mean,days_b_list_min,days_b_list_max,total_trx,mcc_top1_class,mcc_top1_counts,mcc_top1_amount_mean,mcc_top1_amount_std,mcc_top1_amount_max,mcc_top1_amount_min,mcc_top2_class,mcc_top2_counts,mcc_top2_amount_mean,mcc_top2_amount_std,mcc_top2_amount_max,mcc_top2_amount_min,mcc_top3_class,mcc_top3_counts,mcc_top3_amount_mean,mcc_top3_amount_std,mcc_top3_amount_max,mcc_top3_amount_min,mcc_top4_class,mcc_top4_counts,mcc_top4_amount_mean,mcc_top4_amount_std,mcc_top4_amount_max,mcc_top4_amount_min,mcc_top5_class,mcc_top5_counts,mcc_top5_amount_mean,mcc_top5_amount_std,mcc_top5_amount_max,mcc_top5_amount_min,last_mcc,amnt_list_std,amnt_list_mean,amnt_list_min,amnt_list_max,city_top1_class,city_top1_counts,op_kind_top1_class,op_kind_top1_counts,card_type_top1_class,card_type_top1_counts,op_type_top1_class,op_type_top1_counts,payment_system_top1_class,payment_system_top1_counts,income_top1_class,income_top1_counts,city_top2_class,city_top2_counts,op_kind_top2_class,op_kind_top2_counts,card_type_top2_class,card_type_top2_counts,op_type_top2_class,op_type_top2_counts,payment_system_top2_class,payment_system_top2_counts,income_top2_class,income_top2_counts,city_top3_class,city_top3_counts,op_kind_top3_class,op_kind_top3_counts,card_type_top3_class,card_type_top3_counts,op_type_top3_class,op_type_top3_counts,payment_system_top3_class,payment_system_top3_counts,income_top3_class,income_top3_counts,city_top4_class,city_top4_counts,op_kind_top4_class,op_kind_top4_counts,card_type_top4_class,card_type_top4_counts,op_type_top4_class,op_type_top4_counts,payment_system_top4_class,payment_system_top4_counts,income_top4_class,income_top4_counts,city_top5_class,city_top5_counts,last_city,op_kind_top5_class,op_kind_top5_counts,last_op_kind,card_type_top5_class,card_type_top5_counts,op_type_top5_class,op_type_top5_counts,last_op_type,payment_system_top5_class,payment_system_top5_counts,last_payment_system,income_top5_class,income_top5_counts,last_income
0,32,2,7,1,548,63,594,17,0,104.949155,155.441899,1,359,611,4,253,0.306038,0.036479,0.424749,0.192694,1,137,0.329665,0.039944,0.427843,0.229750,3,75,0.310373,0.033390,0.367847,0.220630,2,72,0.424111,0.083555,0.616428,0.235098,9,16,0.399837,0.048436,0.486348,0.268832,3,0.062288,0.333065,0.192694,0.616428,1,465,1,538,11,355,2,296,2,310,1,593,71,62,2,44,7,209,1,236,1,239,2,18,23,41,3,26,84,47,3,29,4,40,-1,-1,163,20,4,3,-1,-1,8,13,6,19,-1,-1,2,14,71,-1,-1,1,-1,-1,6,13,1,3,3,1,-1,-1,1
1,60,2,51,1,135,18,147,1,5,103.809634,185.934641,6,359,153,2,91,0.430963,0.071143,0.572565,0.258972,1,27,0.238406,0.121456,0.392405,0.000000,16,11,0.349390,0.006844,0.359644,0.332881,5,8,0.359847,0.029891,0.437927,0.336345,20,6,0.245631,0.060140,0.377821,0.205577,5,0.090390,0.383106,0.162190,0.572565,132,102,2,91,51,153,7,82,1,144,1,151,2,24,1,55,-1,-1,1,55,6,5,2,2,1,11,6,5,-1,-1,3,9,4,4,-1,-1,96,6,3,1,-1,-1,20,5,-1,-1,-1,-1,37,5,132,5,1,1,-1,-1,10,1,1,-1,-1,1,-1,-1,1
2,90,1,14,1,344,134,475,3,0,100.580562,156.784519,3,358,478,2,305,0.511962,0.094212,0.684959,0.258972,3,36,0.425096,0.035419,0.519181,0.348838,4,26,0.286509,0.059360,0.552579,0.250599,5,20,0.410295,0.065079,0.529560,0.240058,14,20,0.467998,0.084215,0.639389,0.288519,2,0.104905,0.470384,0.240058,0.684959,3,307,2,255,14,400,2,163,2,268,1,475,1,113,1,218,52,62,3,151,4,137,2,3,2,26,3,3,5,16,8,104,6,72,-1,-1,148,8,4,1,-1,-1,16,53,3,1,-1,-1,49,6,49,5,1,2,-1,-1,5,2,8,-1,-1,2,-1,-1,1
3,148,2,14,1,45,0,36,9,0,90.988438,214.355556,4,351,45,2,45,0.504999,0.095371,0.639497,0.258972,-1,-1,-1.000000,-1.000000,-1.000000,-1.000000,-1,-1,-1.000000,-1.000000,-1.000000,-1.000000,-1,-1,-1.000000,-1.000000,-1.000000,-1.000000,-1,-1,-1.000000,-1.000000,-1.000000,-1.000000,2,0.095371,0.504999,0.258972,0.639497,34,45,2,36,52,42,3,36,6,19,1,36,-1,-1,4,9,14,3,4,9,4,17,2,9,-1,-1,-1,-1,-1,-1,-1,-1,3,9,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,34,-1,-1,2,-1,-1,-1,-1,3,

In [9]:
train_df_full = numeric_df.merge(train_df, how="inner", on="app_id")
test_df_full = numeric_df.merge(test_df, how="inner", on="app_id")
del numeric_df, train_df, test_df
gc.collect();

In [10]:
train_df_full = train_df_full.drop(columns=["mcc_cat_list", "amnt_list_discrete", "next_mcc_y"]).rename(columns={"next_mcc_x": "Y"})
test_df_full = test_df_full.drop(columns=["mcc_cat_list", "amnt_list_discrete", "next_mcc_y"]).rename(columns={"next_mcc_x": "Y"})

In [11]:
train_df_full.head(5)

,app_id,Y,last_card_type,last_week,ecomm_1,ecomm_2,optype_gr_1,optype_gr_2,optype_gr_3,days_b_list_std,days_b_list_mean,days_b_list_min,days_b_list_max,total_trx,mcc_top1_class,mcc_top1_counts,mcc_top1_amount_mean,mcc_top1_amount_std,mcc_top1_amount_max,mcc_top1_amount_min,mcc_top2_class,mcc_top2_counts,mcc_top2_amount_mean,mcc_top2_amount_std,mcc_top2_amount_max,mcc_top2_amount_min,mcc_top3_class,mcc_top3_counts,mcc_top3_amount_mean,mcc_top3_amount_std,mcc_top3_amount_max,mcc_top3_amount_min,mcc_top4_class,mcc_top4_counts,mcc_top4_amount_mean,mcc_top4_amount_std,mcc_top4_amount_max,mcc_top4_amount_min,mcc_top5_class,mcc_top5_counts,mcc_top5_amount_mean,mcc_top5_amount_std,mcc_top5_amount_max,mcc_top5_amount_min,last_mcc,amnt_list_std,amnt_list_mean,amnt_list_min,amnt_list_max,city_top1_class,city_top1_counts,op_kind_top1_class,op_kind_top1_counts,card_type_top1_class,card_type_top1_counts,op_type_top1_class,op_type_top1_counts,payment_system_top1_class,payment_system_top1_counts,income_top1_class,income_top1_counts,city_top2_class,city_top2_counts,op_kind_top2_class,op_kind_top2_counts,card_type_top2_class,card_type_top2_counts,op_type_top2_class,op_type_top2_counts,payment_system_top2_class,payment_system_top2_counts,income_top2_class,income_top2_counts,city_top3_class,city_top3_counts,op_kind_top3_class,op_kind_top3_counts,card_type_top3_class,card_type_top3_counts,op_type_top3_class,op_type_top3_counts,payment_system_top3_class,payment_system_top3_counts,income_top3_class,income_top3_counts,city_top4_class,city_top4_counts,op_kind_top4_class,op_kind_top4_counts,card_type_top4_class,card_type_top4_counts,op_type_top4_class,op_type_top4_counts,payment_system_top4_class,payment_system_top4_counts,income_top4_class,income_top4_counts,city_top5_class,city_top5_counts,last_city,op_kind_top5_class,op_kind_top5_counts,last_op_kind,card_type_top5_class,card_type_top5_counts,op_type_top5_class,op_type_top5_counts,last_op_type,payment_system_top5_class,payment_system_top5_counts,last_payment_system,income_top5_class,income_top5_counts,last_income,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255
0,32,2,7,1,548,63,594,17,0,104.949155,155.441899,1,359,611,4,253,0.306038,0.036479,0.424749,0.192694,1,137,0.329665,0.039944,0.427843,0.229750,3,75,0.310373,0.033390,0.367847,0.220630,2,72,0.424111,0.083555,0.616428,0.235098,9,16,0.399837,0.048436,0.486348,0.268832,3,0.062288,0.333065,0.192694,0.616428,1,465,1,538,11,355,2,296,2,310,1,593,71,62,2,44,7,209,1,236,1,239,2,18,23,41,3,26,84,47,3,29,4,40,-1,-1,163,20,4,3,-1,-1,8,13,6,19,-1,-1,2,14,71,-1,-1,1,-1,-1,6,13,1,3,3,1,-1,-1,1,-0.045389,-0.205600,-0.128679,-0.120213,0.156994,0.053706,0.087332,-0.109532,-0.003793,0.103276,-0.128244,-0.069607,-0.051354,-0.180390,0.111737,-0.094453,-0.030913,0.001875,-0.128192,-0.060380,-0.074507,-0.055055,0.023410,0.016163,-0.070287,0.094856,0.176770,-0.244736,0.015249,0.108112,0.243645,0.182034,0.215546,0.100968,0.110819,0.142000,0.131631,0.109563,-0.122887,-0.118338,-0.331012,0.003391,-0.201983,-0.018292,-0.067320,0.065449,-0.014096,0.051266,0.092255,0.017338,0.161364,0.240438,-0.060510,-0.207890,-0.093277,0.0

In [12]:
train_df_full.shape[0] + test_df_full.shape[0]

963800

Имеем немало категориальных переменных, откуда проще будет использовать Catboost:

In [13]:
cat_feats = [i for i in train_df_full.columns if (i.startswith("last_") or "_class" in i) and ("last_week" not in i)]
len(cat_feats)

42

In [14]:
Xtrain, Ytrain = train_df_full.drop(columns=["app_id", "Y"]), train_df_full[["Y"]]
Xtest, Ytest = test_df_full.drop(columns=["app_id", "Y"]), test_df_full[["Y"]]

del train_df_full, test_df_full
gc.collect()

0

In [30]:
model = CatBoostClassifier(
    iterations=256, 
    max_depth=6,
    task_type='GPU',
    devices='0',
    auto_class_weights="SqrtBalanced",
    learning_rate=.02
    )
model.fit(Xtrain, Ytrain, cat_features=cat_feats)

0:	learn: 3.2465935	total: 293ms	remaining: 1m 14s
1:	learn: 3.2068163	total: 594ms	remaining: 1m 15s
2:	learn: 3.1728877	total: 897ms	remaining: 1m 15s
3:	learn: 3.1428441	total: 1.2s	remaining: 1m 15s
4:	learn: 3.1163330	total: 1.48s	remaining: 1m 14s
5:	learn: 3.0897115	total: 1.79s	remaining: 1m 14s
6:	learn: 3.0664679	total: 2.08s	remaining: 1m 13s
7:	learn: 3.0445245	total: 2.4s	remaining: 1m 14s
8:	learn: 3.0239887	total: 2.71s	remaining: 1m 14s
9:	learn: 3.0049733	total: 3.02s	remaining: 1m 14s
10:	learn: 2.9874533	total: 3.31s	remaining: 1m 13s
11:	learn: 2.9703429	total: 3.62s	remaining: 1m 13s
12:	learn: 2.9542197	total: 3.92s	remaining: 1m 13s
13:	learn: 2.9392550	total: 4.24s	remaining: 1m 13s
14:	learn: 2.9232886	total: 4.54s	remaining: 1m 12s
15:	learn: 2.9088622	total: 4.83s	remaining: 1m 12s
16:	learn: 2.8952680	total: 5.12s	remaining: 1m 11s
17:	learn: 2.8823575	total: 5.41s	remaining: 1m 11s
18:	learn: 2.8700768	total: 5.71s	remaining: 1m 11s
19:	learn: 2.8586117	tot

In [ ]:
from sklearn.metrics import top_k_accuracy_score

top_k_accuracy_score

In [20]:
model.predict_proba(Xtest)[3]

array([8.14798090e-02, 7.19114567e-01, 3.34174420e-02, 6.62680247e-03,
       1.22645129e-02, 2.00219540e-02, 2.34945382e-02, 2.33163896e-03,
       1.02196291e-02, 6.73925134e-03, 2.95811791e-02, 3.94119611e-03,
       7.71623545e-03, 5.41959607e-03, 8.40020134e-03, 4.61473529e-03,
       1.95141994e-03, 6.21907450e-03, 2.96150507e-03, 3.10741412e-03,
       3.08214163e-03, 9.66181287e-04, 6.37524646e-04, 3.08317985e-03,
       6.22462891e-04, 1.97696376e-03, 8.84408609e-06])

In [31]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, top_k_accuracy_score

for data, label, name in [(Xtrain, Ytrain, 'Train'), (Xtest, Ytest, 'Test')]:
    preds = model.predict(data)
    print(name)
    # print(classification_report(label, preds))
    print('Acc@5:', top_k_accuracy_score(label, model.predict_proba(data), k=5))
    print('F1 Weighted:', f1_score(label, preds, average='weighted'))
    print('F1 Macro:', f1_score(label, preds, average='macro'))
    print("===")

# 0.8-0.9 Acc@5

Train
Acc@5: 0.8658850383897074
F1 Weighted: 0.47564472467334346
F1 Macro: 0.2198416739353306
===
Test
Acc@5: 0.8627308570242789
F1 Weighted: 0.4723340143847918
F1 Macro: 0.2022401427194106
===


Бейзлайн (самая популярная категория):

In [69]:
Ybaseline_probas = np.zeros((Ytest.shape[0], 27))

for idx in Ytest.value_counts("Y").index[:5].tolist():
    Ybaseline_probas[:, idx-1] = 1

In [70]:
Ybaseline = Ytest.copy()
Ybaseline["Y"] = Ytest.value_counts("Y").index[0]

print("Test baseline - just most popular mcc")
# print(classification_report(Ytest, Ybaseline))
print('Accuracy:', accuracy_score(Ytest, Ybaseline))
print('Acc@5:', top_k_accuracy_score(Ytest, Ybaseline_probas, k=5))
print('F1 Weighted:', f1_score(Ytest, Ybaseline, average='weighted'))
print('F1 Macro:', f1_score(Ytest, Ybaseline, average='macro'))

Test baseline - just most popular mcc
Accuracy: 0.36336376841668394
Acc@5: 0.7922701805353808
F1 Weighted: 0.19368745342456584
F1 Macro: 0.019742225311437114


Откуда, можем сделать вывод, что с кол-вом классов, равным 27, при этом с сильно выраженным неравномерным распределением саппорта, наша модель справляется лучше бейзлайна по Top-5, но заметим и значительное увеличение F1-Macro в Top-1 (приоритизация).

Посмотрим на Feature importance:

In [72]:
pd.DataFrame([{k:v for k, v in zip(model.feature_names_, model.feature_importances_)}]).T.sort_values(0, ascending=False).rename(columns={0: "Gain"}).iloc[:30]

,Gain
last_mcc,9.949379
days_b_list_min,3.880772
220,3.463336
241,2.768477
last_op_kind,2.218884
102,2.118910
225,1.884609
61,1.790324
248,1.533024
237,1.352866
